In [1]:
# Copyright (c) 2024 Microsoft Corporation.
# Licensed under the MIT License.

In [2]:
import os

import pandas as pd
import tiktoken

from graphrag.query.context_builder.entity_extraction import EntityVectorStoreKey
from graphrag.query.indexer_adapters import (
    read_indexer_covariates,
    read_indexer_entities,
    read_indexer_relationships,
    read_indexer_reports,
    read_indexer_text_units,
)
from graphrag.query.input.loaders.dfs import (
    store_entity_semantic_embeddings,
)
from graphrag.query.llm.oai.chat_openai import ChatOpenAI
from graphrag.query.llm.oai.embedding import OpenAIEmbedding
from graphrag.query.llm.oai.typing import OpenaiApiType
from graphrag.query.question_gen.local_gen import LocalQuestionGen
from graphrag.query.structured_search.local_search.mixed_context import (
    LocalSearchMixedContext,
)
from graphrag.query.structured_search.local_search.search import LocalSearch
from graphrag.vector_stores.lancedb import LanceDBVectorStore

## Local Search Example

Local search method generates answers by combining relevant data from the AI-extracted knowledge-graph with text chunks of the raw documents. This method is suitable for questions that require an understanding of specific entities mentioned in the documents (e.g. What are the healing properties of chamomile?).

### Load text units and graph data tables as context for local search

- In this test we first load indexing outputs from parquet files to dataframes, then convert these dataframes into collections of data objects aligning with the knowledge model.

### Load tables to dataframes

In [3]:
INPUT_DIR = "../graphfleet/output/20240827-012901/artifacts"
LANCEDB_URI = f"{INPUT_DIR}/lancedb"

COMMUNITY_REPORT_TABLE = "create_final_community_reports"
ENTITY_TABLE = "create_final_nodes"
ENTITY_EMBEDDING_TABLE = "create_final_entities"
RELATIONSHIP_TABLE = "create_final_relationships"
COVARIATE_TABLE = "create_final_covariates"
TEXT_UNIT_TABLE = "create_final_text_units"
COMMUNITY_LEVEL = 2

#### Read entities

In [4]:
# read nodes table to get community and degree data
entity_df = pd.read_parquet(f"{INPUT_DIR}/{ENTITY_TABLE}.parquet")
entity_embedding_df = pd.read_parquet(f"{INPUT_DIR}/{ENTITY_EMBEDDING_TABLE}.parquet")

entities = read_indexer_entities(entity_df, entity_embedding_df, COMMUNITY_LEVEL)

# load description embeddings to an in-memory lancedb vectorstore
# to connect to a remote db, specify url and port values.
description_embedding_store = LanceDBVectorStore(
    collection_name="entity_description_embeddings",
)
description_embedding_store.connect(db_uri=LANCEDB_URI)
entity_description_embeddings = store_entity_semantic_embeddings(
    entities=entities, vectorstore=description_embedding_store
)

print(f"Entity count: {len(entity_df)}")
entity_df.head()

[2024-08-27T02:28:12Z WARN  lance::dataset] No existing dataset at /workspaces/GraphFleet/notebook/../graphfleet/output/20240827-012901/artifacts/lancedb/entity_description_embeddings.lance, it will be created


Entity count: 11904


,level,title,type,description,source_id,degree,human_readable_id,id,graph_embedding,community,size,entity_type,top_level_node_id,x,y
0,0,DARREN EDGE,PERSON,Darren Edge is one of the authors of the paper...,0c932f7def033fa2b1bf210fbb771e7d,8,0,b45241d70f0e43fca764df95b2b81f77,None,None,NaN,None,b45241d70f0e43fca764df95b2b81f77,NaN,NaN
1,0,HA TRINH,PERSON,Ha Trinh is one of the authors of the paper ti...,0c932f7def033fa2b1bf210fbb771e7d,8,1,4119fd06010c494caa07f439b333f4c5,None,None,NaN,None,4119fd06010c494caa07f439b333f4c5,NaN,NaN
2,0,NEWMAN CHENG,PERSON,Newman Cheng is one of the authors of the pape...,0c932f7def033fa2b1bf210fbb771e7d,8,2,d3835bf3dda84ead99deadbeac5d0d7d,None,None,NaN,None,d3835bf3dda84ead99deadbeac5d0d7d,NaN,NaN
3,0,JOSHUA BRADLEY,PERSON,Joshua Bradley is one of the authors of the pa...,0c932f7def033fa2b1bf210fbb771e7d,8,3,077d2820ae1845bcbb1803379a3d1eae,None,None,NaN,None,077d2820ae1845bcbb1803379a3d1eae,NaN,NaN
4,0,ALEX CHAO,PERSON,Alex Chao is one of the authors of the paper t...,0c932f7def033fa2b1bf210fbb771e7d,8,4,3671ea0dd4e84c1a9b02c5ab2c8f4bac,None,None,NaN,None,3671ea0dd4e84c1a9b02c5ab2c8f4bac,NaN,NaN


#### Read relationships

In [5]:
relationship_df = pd.read_parquet(f"{INPUT_DIR}/{RELATIONSHIP_TABLE}.parquet")
relationships = read_indexer_relationships(relationship_df)

print(f"Relationship count: {len(relationship_df)}")
relationship_df.head()

Relationship count: 2132


,source,target,weight,description,text_unit_ids,id,human_readable_id,source_degree,target_degree,rank
0,DARREN EDGE,HA TRINH,8.0,Darren Edge and Ha Trinh co-authored the paper...,[0c932f7def033fa2b1bf210fbb771e7d],36e157f8abbb4c1fad1ddc80743c0177,0,8,8,16
1,DARREN EDGE,NEWMAN CHENG,8.0,Darren Edge and Newman Cheng co-authored the p...,[0c932f7def033fa2b1bf210fbb771e7d],9ffb31b2ecb848c29aaba26d82d18a6f,1,8,8,16
2,DARREN EDGE,JOSHUA BRADLEY,8.0,Darren Edge and Joshua Bradley co-authored the...,[0c932f7def033fa2b1bf210fbb771e7d],895df887026a4632b8a15e44db6b1306,2,8,8,16
3,DARREN EDGE,ALEX CHAO,8.0,Darren Edge and Alex Chao co-authored the pape...,[0c932f7def033fa2b1bf210fbb771e7d],bca0479aab5e4f58afbb6c9df9cf34a9,3,8,8,16
4,DARREN EDGE,APURVA MODY,8.0,Darren Edge and Apurva Mody co-authored the pa...,[0c932f7def033fa2b1bf210fbb771e7d],e994c254c7234a11ae28e2f1a731007e,4,8,8,16


In [6]:
# NOTE: covariates are turned off by default, because they generally need prompt tuning to be valuable
# Please see the GRAPHRAG_CLAIM_* settings
covariate_df = pd.read_parquet(f"{INPUT_DIR}/{COVARIATE_TABLE}.parquet")

claims = read_indexer_covariates(covariate_df)

print(f"Claim records: {len(claims)}")
covariates = {"claims": claims}

Claim records: 347


#### Read community reports

In [12]:
def read_indexer_entities(entity_df, entity_embedding_df, community_level):
    # Create an explicit copy if needed
    entity_df = entity_df.copy()
    
    # Use .loc to modify the DataFrame
    entity_df.loc[:, "community"] = entity_df["community"].fillna(-1)
    entity_df.loc[:, "community"] = entity_df["community"].astype(int)
    
    # ... rest of the function implementation
    
    return entities  # Assuming this function returns entities

# Outside the function
report_df = pd.read_parquet(f"{INPUT_DIR}/{COMMUNITY_REPORT_TABLE}.parquet")
reports = read_indexer_reports(report_df, entity_df, COMMUNITY_LEVEL)

print(f"Report records: {len(report_df)}")
report_df.head()
    # ... rest of the function

Report records: 239


/workspaces/GraphFleet/.venv/lib/python3.11/site-packages/graphrag/query/indexer_adapters.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  entity_df["community"] = entity_df["community"].fillna(-1)
/workspaces/GraphFleet/.venv/lib/python3.11/site-packages/graphrag/query/indexer_adapters.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  entity_df["community"] = entity_df["community"].astype(int)


,community,full_content,level,rank,title,rank_explanation,summary,findings,full_content_json,id
0,237,# AGIEval and AI Model Performance Evaluation\...,3,8.5,AGIEval and AI Model Performance Evaluation,The rating is high due to AGIEval's significan...,"The community revolves around AGIEval, a human...",[{'explanation': 'AGIEval is a pivotal benchma...,"{\n ""title"": ""AGIEval and AI Model Performa...",c32ecac8-6dea-4fd4-817d-1cd6518f98de
1,238,# Orca-3 and its Ecosystem\n\nThe community re...,3,9.0,Orca-3 and its Ecosystem,The report is highly significant due to the de...,The community revolves around the Orca-3 langu...,[{'explanation': 'Orca-3 has shown notable enh...,"{\n ""title"": ""Orca-3 and its Ecosystem"",\n ...",99902b15-234c-4514-b9db-e04c7ac0f595
2,134,# Agents and Reading Comprehension in AI\n\nTh...,2,8.5,Agents and Reading Comprehension in AI,The rating is high due to the significant impa...,The community focuses on the development and o...,[{'explanation': 'AGENT is a central entity in...,"{\n ""title"": ""Agents and Reading Comprehens...",86999f2f-567c-43ac-809d-d85f8df8092c
3,135,# Seed Instruction Generation Flow and Compreh...,2,8.5,Seed Instruction Generation Flow and Comprehen...,The rating is high due to the significant impa...,The community centers around the 'Seed Instruc...,[{'explanation': 'The Seed Instruction Generat...,"{\n ""title"": ""Seed Instruction Generation F...",73aeb516-a05a-4069-9bed-0f365a739667
4,136,# Instruction Refinement Flow in AgentInstruct...,2,8.5,Instruction Refinement Flow in AgentInstruct,The rating is high due to the significant impa...,The community centers around the Instruction R...,[{'explanation': 'The Instruction Refinement F...,"{\n ""title"": ""Instruction Refinement Flow i...",054de93a-962b-4a11-8c95-44ac2c350e15


In [13]:
report_df = pd.read_parquet(f"{INPUT_DIR}/{COMMUNITY_REPORT_TABLE}.parquet")
reports = read_indexer_reports(report_df, entity_df, COMMUNITY_LEVEL)

print(f"Report records: {len(report_df)}")
report_df.head()

Report records: 239


/workspaces/GraphFleet/.venv/lib/python3.11/site-packages/graphrag/query/indexer_adapters.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  entity_df["community"] = entity_df["community"].fillna(-1)
/workspaces/GraphFleet/.venv/lib/python3.11/site-packages/graphrag/query/indexer_adapters.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  entity_df["community"] = entity_df["community"].astype(int)


,community,full_content,level,rank,title,rank_explanation,summary,findings,full_content_json,id
0,237,# AGIEval and AI Model Performance Evaluation\...,3,8.5,AGIEval and AI Model Performance Evaluation,The rating is high due to AGIEval's significan...,"The community revolves around AGIEval, a human...",[{'explanation': 'AGIEval is a pivotal benchma...,"{\n ""title"": ""AGIEval and AI Model Performa...",c32ecac8-6dea-4fd4-817d-1cd6518f98de
1,238,# Orca-3 and its Ecosystem\n\nThe community re...,3,9.0,Orca-3 and its Ecosystem,The report is highly significant due to the de...,The community revolves around the Orca-3 langu...,[{'explanation': 'Orca-3 has shown notable enh...,"{\n ""title"": ""Orca-3 and its Ecosystem"",\n ...",99902b15-234c-4514-b9db-e04c7ac0f595
2,134,# Agents and Reading Comprehension in AI\n\nTh...,2,8.5,Agents and Reading Comprehension in AI,The rating is high due to the significant impa...,The community focuses on the development and o...,[{'explanation': 'AGENT is a central entity in...,"{\n ""title"": ""Agents and Reading Comprehens...",86999f2f-567c-43ac-809d-d85f8df8092c
3,135,# Seed Instruction Generation Flow and Compreh...,2,8.5,Seed Instruction Generation Flow and Comprehen...,The rating is high due to the significant impa...,The community centers around the 'Seed Instruc...,[{'explanation': 'The Seed Instruction Generat...,"{\n ""title"": ""Seed Instruction Generation F...",73aeb516-a05a-4069-9bed-0f365a739667
4,136,# Instruction Refinement Flow in AgentInstruct...,2,8.5,Instruction Refinement Flow in AgentInstruct,The rating is high due to the significant impa...,The community centers around the Instruction R...,[{'explanation': 'The Instruction Refinement F...,"{\n ""title"": ""Instruction Refinement Flow i...",054de93a-962b-4a11-8c95-44ac2c350e15


#### Read text units

In [14]:
text_unit_df = pd.read_parquet(f"{INPUT_DIR}/{TEXT_UNIT_TABLE}.parquet")
text_units = read_indexer_text_units(text_unit_df)

print(f"Text unit records: {len(text_unit_df)}")
text_unit_df.head()

Text unit records: 82


,id,text,n_tokens,document_ids,entity_ids,relationship_ids,covariate_ids
0,0c932f7def033fa2b1bf210fbb771e7d,From Local to Global: A Graph RAG Approach to\...,1200,[0668cddc5f873265ba50da5a0a06edad],"[b45241d70f0e43fca764df95b2b81f77, 4119fd06010...","[36e157f8abbb4c1fad1ddc80743c0177, 9ffb31b2ecb...","[2222cb5c-1189-4849-a653-a87da5386de1, 8c1c5dc..."
1,64476a39d7d8b87b399e3bd3cead79c7,on\nthe state-of-the-art for all such summari...,1200,[0668cddc5f873265ba50da5a0a06edad],"[de988724cfdf45cebfba3b13c43ceede, d91a266f766...","[946854329dad408381d64bc8bee762b1, bd219396154...","[262d01ec-636a-4c6a-9731-e6934bd905ef, ff3a5bb..."
2,e66ed885a08f92cc69f4895302c33047,examples provided to the LLM for in-context l...,1200,[0668cddc5f873265ba50da5a0a06edad],"[d91a266f766b4737a06b0fda588ba40b, de9e343f2e3...","[67520fc5fcd646769bd13af2264c9d46, 5511a12ded6...",[67f01e9a-f142-48c1-a537-a5e04be28ff5]
3,4930fce6da868f894757a9da465807ba,which reveals internal structure within these...,1200,[0668cddc5f873265ba50da5a0a06edad],"[deece7e64b2a4628850d4bb6e394a9c3, b7702b90c7f...","[7146e1a5f4a949aa8802f82c07245a51, 78f01eb7e2f...",[184e5780-04f2-4fbf-8697-375fd66e811f]
4,26b2dad01a219bc034ac7d6a32d07582,"understanding of dataset contents, and not th...",1200,[0668cddc5f873265ba50da5a0a06edad],"[de988724cfdf45cebfba3b13c43ceede, 2670deebfa3...","[807b66ed814e4b25a499ccb80afd56c5, 6ed5ba686d9...","[c36e328f-78fb-4b59-89ab-caa9ceea918f, 523bae4..."


In [15]:
api_key = os.environ["GRAPHRAG_API_KEY"]
llm_model = os.environ["GRAPHRAG_LLM_MODEL"]
embedding_model = os.environ["GRAPHRAG_EMBEDDING_MODEL"]
api_base = os.environ["GRAPHRAG_API_BASE"]
api_version = os.environ["GRAPHRAG_API_VERSION"]


llm = ChatOpenAI(
    api_key=api_key,
    api_base=api_base,
    api_version=api_version,
    model=llm_model,
    api_type=OpenaiApiType.AzureOpenAI,  # OpenaiApiType.OpenAI or OpenaiApiType.AzureOpenAI
    max_retries=20,
)

token_encoder = tiktoken.get_encoding("cl100k_base")

text_embedder = OpenAIEmbedding(
    api_key=api_key,
    api_base=api_base,
    api_version=api_version,
    api_type=OpenaiApiType.AzureOpenAI,
    model=embedding_model,
    deployment_name=embedding_model,
    max_retries=20,
)

### Create local search context builder

In [16]:
context_builder = LocalSearchMixedContext(
    community_reports=reports,
    text_units=text_units,
    entities=entities,
    relationships=relationships,
    # if you did not run covariates during indexing, set this to None
    covariates=covariates,
    entity_text_embeddings=description_embedding_store,
    embedding_vectorstore_key=EntityVectorStoreKey.ID,  # if the vectorstore uses entity title as ids, set this to EntityVectorStoreKey.TITLE
    text_embedder=text_embedder,
    token_encoder=token_encoder,
)

### Create local search engine

In [17]:
# text_unit_prop: proportion of context window dedicated to related text units
# community_prop: proportion of context window dedicated to community reports.
# The remaining proportion is dedicated to entities and relationships. Sum of text_unit_prop and community_prop should be <= 1
# conversation_history_max_turns: maximum number of turns to include in the conversation history.
# conversation_history_user_turns_only: if True, only include user queries in the conversation history.
# top_k_mapped_entities: number of related entities to retrieve from the entity description embedding store.
# top_k_relationships: control the number of out-of-network relationships to pull into the context window.
# include_entity_rank: if True, include the entity rank in the entity table in the context window. Default entity rank = node degree.
# include_relationship_weight: if True, include the relationship weight in the context window.
# include_community_rank: if True, include the community rank in the context window.
# return_candidate_context: if True, return a set of dataframes containing all candidate entity/relationship/covariate records that
# could be relevant. Note that not all of these records will be included in the context window. The "in_context" column in these
# dataframes indicates whether the record is included in the context window.
# max_tokens: maximum number of tokens to use for the context window.


local_context_params = {
    "text_unit_prop": 0.5,
    "community_prop": 0.1,
    "conversation_history_max_turns": 5,
    "conversation_history_user_turns_only": True,
    "top_k_mapped_entities": 10,
    "top_k_relationships": 10,
    "include_entity_rank": True,
    "include_relationship_weight": True,
    "include_community_rank": False,
    "return_candidate_context": False,
    "embedding_vectorstore_key": EntityVectorStoreKey.ID,  # set this to EntityVectorStoreKey.TITLE if the vectorstore uses entity title as ids
    "max_tokens": 12_000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 5000)
}

llm_params = {
    "max_tokens": 2_000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 1000=1500)
    "temperature": 0.0,
}

In [18]:
search_engine = LocalSearch(
    llm=llm,
    context_builder=context_builder,
    token_encoder=token_encoder,
    llm_params=llm_params,
    context_builder_params=local_context_params,
    response_type="multiple paragraphs",  # free form text describing the response type and format, can be anything, e.g. prioritized list, single paragraph, multiple paragraphs, multiple-page report
)

### Run local search on sample queries

In [19]:
result = await search_engine.asearch("What is GraphRAG be short")
print(result.response)

### Overview of GraphRAG

GraphRAG is an advanced system designed to enhance retrieval-augmented generation (RAG) by leveraging the modularity of graphs and community detection algorithms. It builds a graph-based text index using large language models (LLMs) to improve the efficiency and comprehensiveness of information retrieval and summarization [Data: Graph RAG and Community Detection in AI and ML (77)].

### Key Features

1. **Superior Performance**: GraphRAG outperforms Naïve RAG in terms of comprehensiveness and diversity of answers by partitioning graphs into modular communities [Data: Graph RAG and Community Detection in AI and ML (77)].
2. **Integration with LLMs**: It uses LLMs to create a graph-based text index, facilitating precise and contextually relevant information retrieval [Data: Graph RAG and Community Detection in AI and ML (77)].
3. **Scalability**: The system is highly scalable, capable of handling large text corpora and complex queries efficiently [Data: Graph RA

In [20]:
question = "What is the purpose of GraphRAG?"
result = await search_engine.asearch(question)
print(result.response)

# Purpose of GraphRAG

GraphRAG is designed to enhance the process of retrieval-augmented generation (RAG) by leveraging the structure and modularity of knowledge graphs. The primary purpose of GraphRAG is to improve the efficiency and accuracy of information retrieval and summarization, particularly for complex queries over large text corpora.

## Enhancing Information Retrieval

GraphRAG utilizes community detection algorithms to partition graphs into modular communities. This approach allows for more efficient and accurate information retrieval by focusing on closely related nodes within the graph. By doing so, GraphRAG can generate more comprehensive and diverse answers compared to traditional RAG systems [Data: Graph RAG and Community Detection in AI and ML (77); Entities (11); Claims (46, 48)].

## Integration with Large Language Models

One of the key features of GraphRAG is its integration with large language models (LLMs). This integration enables the system to build a graph-b

#### Inspecting the context data used to generate the response

In [21]:
result.context_data["entities"].head()

,id,entity,description,number of relationships,in_context
0,11,GRAPH RAG,Graph RAG is a method that leverages a graph-b...,30,True
1,247,GRAPHRAG,GraphRAG is a system designed to create and re...,2,True
2,282,GRAPH-BASED RAG APPLICATIONS,Graph-based RAG applications are systems that ...,4,True
3,57,GRAPH RAG APPROACH,Graph RAG Approach is a method that involves t...,2,True
4,246,NEBULA-GRAPH,Nebula-Graph is a graph database format suppor...,1,True


In [22]:
result.context_data["relationships"].head()

,id,source,target,description,weight,rank,links,in_context
0,82,GRAPH RAG,GRAPH-BASED RAG APPLICATIONS,Graph RAG is an approach within the broader ca...,9.0,34,1,True
1,58,GRAPH RAG,KNOWLEDGE GRAPH,Graph RAG approach is based on the global summ...,9.0,31,1,True
2,60,GRAPH RAG,ACTIVITY-CENTERED SENSE-MAKING QUESTIONS,Activity-centered sense-making questions are u...,7.0,31,1,True
3,77,GRAPH RAG,KAPING,Graph RAG is related to KAPING where the index...,10.0,31,1,True
4,78,GRAPH RAG,G-RETRIEVER,Graph RAG is related to G-Retriever where subs...,10.0,31,1,True


In [23]:
result.context_data["reports"].head()

,id,title,content
0,77,Graph RAG and Community Detection in AI and ML,# Graph RAG and Community Detection in AI and ...
1,77,Graph RAG and Community Detection in AI and ML,# Graph RAG and Community Detection in AI and ...


In [24]:
result.context_data["sources"].head()

,id,text
0,7,"win rates of 57% and 64%, respectively. Diver..."
1,0,From Local to Global: A Graph RAG Approach to\...
2,1,on\nthe state-of-the-art for all such summari...
3,4,"understanding of dataset contents, and not th..."


In [25]:
if "claims" in result.context_data:
    print(result.context_data["claims"].head())

   id     entity object_id status           start_date             end_date  \
0  34  GRAPH RAG      NONE   TRUE                 NONE                 NONE   
1  45  GRAPH RAG      NONE   TRUE  2024-01-01T00:00:00  2024-12-31T00:00:00   
2  46  GRAPH RAG      NONE   TRUE  2024-01-01T00:00:00  2024-12-31T00:00:00   
3  47  GRAPH RAG      NONE   TRUE  2024-01-01T00:00:00  2024-12-31T00:00:00   
4  48  GRAPH RAG      NONE   TRUE  2024-01-01T00:00:00  2024-12-31T00:00:00   

                                         description  in_context  
0  Graph RAG uses a graph index created with gene...        True  
1  Graph RAG is mentioned as a system that uses t...        True  
2  Graph RAG is mentioned to achieve the best hea...        True  
3  Graph RAG offers many possibilities for refine...        True  
4  Graph RAG shows substantial improvements over ...        True  


### Question Generation

This function takes a list of user queries and generates the next candidate questions.

In [26]:
question_generator = LocalQuestionGen(
    llm=llm,
    context_builder=context_builder,
    token_encoder=token_encoder,
    llm_params=llm_params,
    context_builder_params=local_context_params,
)

In [27]:
question_history = [
    "Tell me about Language Agent Tree Search?",
    "What is the best way to create a knowledge graph agent ?",
]
candidate_questions = await question_generator.agenerate(
    question_history=question_history, context_data=None, question_count=5
)
print(candidate_questions.response)

['- What are the key components of the Language Agent Tree Search (LATS) framework?', '- How does LATS integrate Monte Carlo Tree Search (MCTS) to enhance decision-making?', '- What datasets are used to evaluate the performance of LATS?', '- Who are the authors of the paper "Language Agent Tree Search Unifies Reasoning, Acting, and Planning in Language Models"?', '- How does LATS compare to other methods like ReAct and CoT in terms of performance and efficiency?']
